# Kaggle Neuroblastoma Detection
## Lachlan Dryburgh 2021

Tensorflow implementation of a u-net image segmentation convolutional nerual network.  Trained to label neurons, astrocytes and neuroglioblastoma cell in microscope images.

## Imports and Defines

In [78]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os


IMG_HEIGHT = 540
IMG_WIDTH = 704
NUM_CLASS = 3

CELL = {
    'shsy5y':0,
    'astro':1,
    'cort':2
}

## Importing Images

In [29]:
ids = list(set(df['id']))

len(ids)

In [15]:
train_img = "../input/sartorius-cell-instance-segmentation/train"
train_csv = "../input/sartorius-cell-instance-segmentation/train.csv"

df = pd.read_csv(train_csv)
df.head()


In [152]:
import random

random_id = random.choice(ids)

sample_path = f"../input/sartorius-cell-instance-segmentation/train/{random_id}.png"


im = plt.imread(sample_path)
plt.suptitle(random_id,fontweight="bold", size=20)
plt.imshow(im, cmap = 'seismic')

## Image pixel annotation mask 

In [159]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)

def generate_mask(id, shape):
    mask = np.zeros((shape[0], shape[1], shape[2]), dtype=bool)
    
    for index, row in df[df['id']==id].iterrows():
        
        c = CELL[row['cell_type']]
        
        m = rle_decode(row['annotation'], (IMG_HEIGHT, IMG_WIDTH))
        
        mask[:,:,c] += np.array(m, dtype=bool)
        mask = mask.clip(0,1)
       
    return mask

In [153]:
m = generate_mask(random_id, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS))

print(random_id)

figure, ax = plt.subplots(2,figsize=(20,20))
plt.suptitle(random_id,fontweight="bold", size=20)
ax[0].imshow(im, cmap = 'seismic')
ax[1].imshow(np.array(m, dtype=np.float32))

In [190]:
shsy5y = '1c4f14cce8ee'
astro = '129f894abe35'
cort = '95de75855f80'

s_path = f"../input/sartorius-cell-instance-segmentation/train/{shsy5y}.png"
a_path = f"../input/sartorius-cell-instance-segmentation/train/{astro}.png"
c_path = f"../input/sartorius-cell-instance-segmentation/train/{cort}.png"

s_im = plt.imread(s_path)
a_im = plt.imread(a_path)
c_im = plt.imread(c_path)

figure, ax = plt.subplots(2,3,figsize=(15,9))
plt.suptitle("Images and Masks",fontweight="bold", size=20)

ax[0,0].imshow(s_im, cmap = 'seismic')
ax[0,0].set_title(f"SH-SY5Y  - {shsy5y}")
ax[1,0].imshow(np.array(generate_mask(shsy5y, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS)), dtype=np.float32))
ax[1,0].set_title(f"{shsy5y} annotation mask")
ax[0,1].imshow(a_im, cmap = 'seismic')
ax[0,1].set_title(f"Astrocyte - {astro}")
ax[1,1].imshow(np.array(generate_mask(astro, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS)), dtype=np.float32))
ax[1,1].set_title(f"{astro} annotation mask")
ax[0,2].imshow(c_im, cmap = 'seismic')
ax[0,2].set_title(f"Coritical Neuron  - {cort}")
ax[1,2].imshow(np.array(generate_mask(cort, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS)), dtype=np.float32))
ax[1,2].set_title(f"{cort} annotation mask")
[axi.set_axis_off() for axi in ax.ravel()]
figure.tight_layout()
figure.show()


## Define out model
We are actually defining 2 models.

The downstack of the u-net is used as image classifier so that it can be trained where we only have labels for the enire image rather than pixels.  This will allow transfer learning.

The Downstack feeds back into the upstack for our unet pixel classifier.


In [ ]:
in1 = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))

conv1 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
conv1 = layers.Dropout(0.2)(conv1)
conv1 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

conv2 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
conv2 = layers.Dropout(0.2)(conv2)
conv2 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2)

conv3 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
conv3 = layers.Dropout(0.2)(conv3)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
pool3 = layers.MaxPooling2D((2, 2))(conv3)

conv4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
conv4 = layers.Dropout(0.2)(conv4)
conv4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

up1 = layers.UpSampling2D((2, 2))(conv4)
pad = layers.ZeroPadding2D(padding=((1,0),(0,0)))(up1)
up1 = layers.concatenate([pad, conv3], axis=-1)
conv5 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
conv5 = layers.Dropout(0.2)(conv5)
conv5 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)

up2 = layers.concatenate([layers.UpSampling2D((2, 2))(conv5), conv2], axis=-1)
conv6 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
conv6 = layers.Dropout(0.2)(conv6)
conv6 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

up3 = layers.concatenate([layers.UpSampling2D((2, 2))(conv6), conv1], axis=-1)
conv7 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up3)
conv7 = layers.Dropout(0.2)(conv7)
conv7 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
segmentation = layers.Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv7)

class_box = layers.Flatten()(conv4)
class_box = layers.Dense(128, activation = 'relu')(class_box)
class_box = layers.Dense(NUM_CLASS, activation = 'softmax')(class_box)

class_model = keras.Model(inputs=[in1], outputs=[class_box])

model = keras.Model(inputs=[in1], outputs=[segmentation])



## Compile the class-box model


In [ ]:
class_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
keras.utils.plot_model(class_model, "downstack.png", show_shapes=True)

## Compile the full model

In [ ]:
losses = {'seg': 'sparse_categorical_crossentropy'
            }

metrics = {'seg': ['acc']
                }
model.compile(optimizer="adam", loss = losses, metrics=metrics)

In [ ]:
keras.utils.plot_model(model, "full_model.png", show_shapes=True)